# ANT404 Lab #2: Query CloudTrail Nested JSON

In this lab you will use Redshift Spectrum to query AWS CloudTrail logs. A bucket of example logs has been shared with your test account.

**Background AWS CloudTrails Logs**

AWS CloudTrail records activity on your AWS account and delivers log files to your Amazon S3 bucket. 

CloudTrail records important information about each action, including who made the request, the services used, the actions performed, parameters for the actions, and the response elements returned by the AWS service. 

CloudTrail logs are `JSON` data with nested data elements
* Use the nested data syntax to define a table on the JSON data
* Verify external table is working and query totals are correct
* Review the table in the AWS Glue data catalog

In this lab you will use Amazon Redshift Spectrum to query AWS CloudTrail logs. A bucket of example logs has been shared with your test account. The logs are `JSON` formatted data with nested data elements.

Redshift Spectrum supports querying nested data in `Parquet`, `ORC`, `JSON`, and `Ion` file formats. You will use the nested data `DDL` syntax to define a table on the AWS CloudTrail logs.

## 1. Check for credentials file
Check for the credntials created in the `START_HERE` notebook.

In [1]:
%%bash
cat ant404-lab.creds

{
  "user_name": "ant404",
  "password": "Pp-86feeb76",
  "host_name": "10.0.54.136",
  "port_num": "5439",
  "db_name": "dev"
}


## 2. Set local variables from credentials file
Run this `cell` to import the credentials created in `START_HERE` notebook into this notebook. Later cells rely on these variables.

In [2]:
import simplejson
with open("ant404-lab.creds") as fh:
    creds = simplejson.loads(fh.read())
username=creds["user_name"]
password=creds["password"]
host_name=creds["host_name"]
port_num=creds["port_num"]
db_name=creds["db_name"]

# Example Account, Region, and Cluster values for this lab
account=123456789101
region="us-east-1"
cluster_name="reporting-cluster"

# Default date values used to get sample files
log_year=2019
log_month=11
log_day=10 

## 3. Set `env` shell variables with CloudTrail log location elements
Run the `cell` to set these variables in the local shell. Do not quote the `%set_env` variable strings.

In [3]:
%set_env username={username}
%set_env account={account}

# Default date value used to get sample files
%set_env log_date={log_year}{log_month}{log_day}
%set_env log_date_path=year={log_year}/month={log_month}/day={log_day}

# S3 bucket for logs 
%set_env bucket=redshift-managed-spectrum-datasets-{region}

# S3 prefix path for logs
%set_env log_prefix=dataset=cloudtrail/region={region}
# %set_env audit_prefix=cluster-audit-logs/AWSLogs/{account}/redshift/{region}

# Log file name excluding date
%set_env log_file={account}_CloudTrail_{region}_

env: username=ant404
env: account=123456789101
env: log_date=20191110
env: log_date_path=year=2019/month=11/day=10
env: bucket=redshift-managed-spectrum-datasets-us-east-1
env: log_prefix=dataset=cloudtrail/region=us-east-1
env: log_file=123456789101_CloudTrail_us-east-1_


## 4. List the CloudTrail log files
Run the following `cell` to see a list of the first 15 CloudTrail log files in the specified S3 location

In [4]:
%%bash
# logpath=$bucket/$log_prefix/$log_date/$log_file
# echo "log files: " $(aws s3 ls s3://$logpath | wc -l)
# aws s3 ls s3://$logpath --human-readable | head -15
log_full_prefix=$bucket/$log_prefix/$log_date_path/$log_file

## Count and List the logs with the AWS CLI
echo "CloudTrail files: "
echo "s3://$log_full_prefix"
echo "-----------------------------"
aws s3 ls s3://$log_full_prefix --summarize | tail -15


CloudTrail files: 
s3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=10/123456789101_CloudTrail_us-east-1_
-----------------------------
2019-11-23 03:04:03       2410 123456789101_CloudTrail_us-east-1_20191110T2345Z_oLsrmUpSQJmbO0FU.json.gz
2019-11-23 03:04:03      17165 123456789101_CloudTrail_us-east-1_20191110T2350Z_4bDyzDzErXPf3LZH.json.gz
2019-11-23 03:04:03      29920 123456789101_CloudTrail_us-east-1_20191110T2350Z_6T7DePqM42RvbR4S.json.gz
2019-11-23 03:04:03      27432 123456789101_CloudTrail_us-east-1_20191110T2350Z_GA0rwUEe0IZTAeUl.json.gz
2019-11-23 03:04:03      25419 123456789101_CloudTrail_us-east-1_20191110T2350Z_IPm1w5JkXz7YlBSl.json.gz
2019-11-23 03:04:04      18463 123456789101_CloudTrail_us-east-1_20191110T2350Z_Q3S65miz3Fwr3sNt.json.gz
2019-11-23 03:04:04       1561 123456789101_CloudTrail_us-east-1_20191110T2350Z_koa8bosPcPyf46TD.json.gz
2019-11-23 03:04:04      27668 123456789101_CloudTrail_us-east-1_2019

## 5. Download the one file to the notebook server 

In [ ]:
%%bash
logpath=$bucket/$log_prefix/$log_date_path
file=$log_file$log_date'T2355Z_zrNOAxluDDWrtSga.json.gz'
aws s3 cp s3://$logpath/${file} ${file}
gzip -df ${file}

## 6. Preview the file to determine the data structure

Redshift CloudTrail logs use a complex JSON format with nested data elements.

In [ ]:
import json
with open("123456789101_CloudTrail_us-east-1_20191110T2355Z_zrNOAxluDDWrtSga.json", 'r') as handle:
    parsed = json.load(handle)
# Preview the file
print(json.dumps(parsed, indent=4, sort_keys=True))

## 7. Identify nested data elements
There are 2 key types of nested data elements
* Scalar objects `{ }` where each field can be accessed with dot notation directly
* Inline arrays `[ ]` which need to be declared as a table in your query to be accessed

### Scalar Objects
The `userIdentity` object is **scalar** because it contains no repeating elements. Fields are accessed directly using dot notation.  
```json
"userIdentity": {
  "type": "AssumedRole",
  "accountId": "123456789101",
  "sessionContext": {
    "attributes": {
      "mfaAuthenticated": "false",
      "creationDate": "2019-11-12T18:30:46Z" },
    "sessionIssuer": {
      "type": "Role",
      "arn": "arn:aws:iam::123456789101:role/EC2_InternalSite",
      "accountId": "123456789101",
      "userName": "EC2_InternalSite" }
  }
},
```

### Inline Arrays
By contrast, the data element `resources` contains an **array** of JSON objects. Fields are accessed by declaring the array as a sub-table in the `FROM` clause using PartiQL syntax.
```json
"resources": [
   { "ARN": "arn:aws:dynamodb:us-west-2:123456789101:table",
     "accountId": "123456789101",
     "type": "AWS::DynamoDB::Table" },
   { "ARN": "arn:aws:kms:us-west-2:123456789101:key",
     "accountId": "123456789101",
     "type": "AWS::KMS::Key" },
   { "ARN": "arn:aws:iam:us-west-2:123456789101:role",
     "accountId": "123456789101",
     "type": "AWS::IAM::Role" }
 ],
```

## 8. Connect to your Redshift cluster

You will use the `sqlalchemy` and `ipython-sql` Python libraries to manage the Redshift connection. 

This cell creates a `%sql` element so we can use the connection in other cells in the notebook.

-------
_**Note:** Please ignore the pink error message that says: "UserWarning: The psycopg2 wheel package will be renamed from release 2.8"_'**Look for** 'Connected: ant404@dev' in the 'Out [ ]' section below the warning.

In [ ]:
import sqlalchemy
import psycopg2
import simplejson

%reload_ext sql
%config SqlMagic.displaylimit = 25

connect_to_db = 'postgresql+psycopg2://'+username+':'+password+'@'+host_name+':'+port_num+'/'+db_name
%sql $connect_to_db

## 9. List existing external `database`/`schema`/`table`
These tables should now contain the tables created in Lab #1
 

In [ ]:
%sql SELECT * FROM svv_external_databases WHERE databasename = 'logdata';

In [ ]:
%sql SELECT * FROM svv_external_schemas WHERE schemaname = 'rawlogs';

In [ ]:
%sql SELECT * FROM svv_external_tables WHERE schemaname = 'rawlogs' ORDER BY schemaname, tablename;

## 10. Compose the external table DDL using nested data syntax
External tables that reference nested JSON data use the following data types:

* **Objects**
    * JSON - `{"field_1": "value_1", "field_2": 2}`
    * DDL - `struct< keyname:datatype [,keyname:datatype] >`
* **Arrays**
    * JSON - `[\"value_1\",\"value_2\"]`
    * DDL - `array< datatype >`

### Scalar Object DDL
Each scalar object is declared as a `STRUCT < >` in Redshift Spectrum's nested data DDL syntax.
```sql
userIdentity        STRUCT <
  type:               VARCHAR,
  accountId:          VARCHAR,
  sessionContext:       STRUCT <
    attributes:           STRUCT <
      mfaAuthenticated:     VARCHAR,
      creationDate:         VARCHAR >,
    sessionIssuer:        STRUCT <
      type:                 VARCHAR,
      arn:                  VARCHAR,
      accountId:            VARCHAR,
      userName:               VARCHAR > > >,
```

### Inline Array DDL
```sql
resources     ARRAY< STRUCT<
  arn:          VARCHAR,
  accountId:    VARCHAR,
  type:         VARCHAR > >,
```

The first section of your DDL should look like this:
```sql
CREATE EXTERNAL TABLE rawlogs.cloudtrail (
    records             ARRAY<
```


Use the provided DDL to create an external table for this data set.  
**Note that this DDL uses the external schema and database you created in Lab #1, Step 10.**

In [ ]:
%%sql 
/* -- Escape autocommit with */END;/* -- */
DROP TABLE IF EXISTS rawlogs.cloudtrail;
CREATE EXTERNAL TABLE rawlogs.cloudtrail (
    records             ARRAY< STRUCT<
        eventversion:       VARCHAR(8),
        useridentity:       STRUCT<
            type:                   VARCHAR(16),
            principalid:            VARCHAR(128),
            arn:                    VARCHAR(256),
            accountid:              VARCHAR(16),
            invokedby:              VARCHAR(64),
            accesskeyid:            VARCHAR(32),
            username:               VARCHAR(32),
            sessioncontext:         STRUCT<
                attributes:             STRUCT<
                    mfaauthenticated:       VARCHAR(8),
                    creationdate:           VARCHAR(32)>,
                sessionissuer:          STRUCT<
                    type:                   VARCHAR(8),
                    principalid:            VARCHAR(32),
                    arn:                    VARCHAR(256),
                    accountid:              VARCHAR(16),
                    username:                   VARCHAR(64) > > >,
        eventtime:           VARCHAR(32),
        eventsource:         VARCHAR(64),
        eventname:           VARCHAR(64),
        awsregion:           VARCHAR(16),
        sourceipaddress:     VARCHAR(64),
        useragent:           VARCHAR(256),
        errorcode:           VARCHAR(64),
        errormessage:        VARCHAR(512),
        requestparameters:   STRUCT<
            durationseconds:    INTEGER,
            rolearn:            VARCHAR(256),
            rolesessionname:    VARCHAR(64),
            partitionValues:    ARRAY<VARCHAR>,
            databaseName:       VARCHAR(16),
            tableName:          VARCHAR(64) >,
        responseelements:    STRUCT<
            assumedRoleUser:    STRUCT<
                arn:                VARCHAR(128),
                assumedRoleId:      VARCHAR(64) >,
            "credentials":      STRUCT<
                accessKeyId:        VARCHAR(32),
                expiration:         VARCHAR(32),
                sessionToken:       VARCHAR(2048) > >,
        additionaleventdata: STRUCT<
            insufficientLakeFormationPermissions: ARRAY<VARCHAR >,
            lakeFormationPrincipal: VARCHAR(128) >,
        requestid:           VARCHAR(64),
        eventid:             VARCHAR(64),
        resources:           ARRAY< STRUCT<
            arn:                           VARCHAR(256),
            accountid:                     VARCHAR(16),
            type:                          VARCHAR(32) > >,
        eventtype:           VARCHAR(32),
        apiversion:          VARCHAR(16),
        readonly:            VARCHAR(8),
        recipientaccountid:  VARCHAR(16),
        serviceeventdetails: VARCHAR(1024),
        sharedeventid:       VARCHAR(64),
        vpcendpointid:       VARCHAR(16) > >
) 
PARTITIONED BY (
      region VARCHAR(32)
    , log_year INT
    , log_month INT
    , log_day INT
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/cloudtrail/'
TABLE PROPERTIES ('compression_type'='gzip')
;
SELECT * FROM svv_external_tables WHERE schemaname = 'rawlogs' AND tablename = 'cloudtrail';

## 11. Add partitions for each day
There is no data associated with a partitioned table until at least one partition is added.

In [ ]:
%%sql
/* -- Escape autocommit with */END;/* -- */
ALTER TABLE rawlogs.cloudtrail 
ADD IF NOT EXISTS
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=1 ) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=01/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=2 ) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=02/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=3 ) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=03/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=4 ) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=04/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=5 ) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=05/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=6 ) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=06/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=7 ) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=07/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=8 ) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=08/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=9 ) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=09/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=10) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=10/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=11) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=11/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=12) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=12/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=13) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=13/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=14) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=14/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=15) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=15/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=16) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=16/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=17) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=17/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=18) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=18/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=19) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=19/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=20) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=20/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=21) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=21/'
    PARTITION (region='us-east-1', log_year=2019, log_month=11, log_day=22) LOCATION 's3://redshift-managed-spectrum-datasets-us-east-1/dataset=cloudtrail/region=us-east-1/year=2019/month=11/day=22/'
;
SELECT * FROM svv_external_partitions WHERE schemaname = 'rawlogs' AND tablename = 'cloudtrail' ;

## 12. Verify external table works - Partition Columns

In [ ]:
%%sql 
SELECT rec.eventSource 
     , log.region 
     , COUNT(*) 
FROM rawlogs.cloudtrail log 
   , log.records rec 
WHERE log.log_day = 9
GROUP BY 1,2 
ORDER BY 1,2;

## 13. Verify external table works - Scalar Object Queries
The struct fields are accessed directly using dot notation. For example, the following query accesses the fields from the `userIdentity` object.

In [ ]:
%%sql 
SELECT rec.userIdentity.type
     , rec.userIdentity.principalId
     , rec.userIdentity.arn
     , rec.userIdentity.accountId
     , rec.userIdentity.invokedBy
     , rec.userIdentity.accessKeyId
     , rec.userIdentity.userName
     , rec.userIdentity.sessionContext.attributes.mfaAuthenticated
     , rec.userIdentity.sessionContext.attributes.creationDate
     , rec.userIdentity.sessionContext.sessionIssuer.type
     , rec.userIdentity.sessionContext.sessionIssuer.principalId
     , rec.userIdentity.sessionContext.sessionIssuer.arn
     , rec.userIdentity.sessionContext.sessionIssuer.accountId
     , rec.userIdentity.sessionContext.sessionIssuer.userName
FROM rawlogs.cloudtrail log 
   , log.records rec -- # Top level inline "Records" array
WHERE log.region = 'us-east-1' 
  AND log.log_year = 2019 
  AND log.log_month = 11 
  AND log.log_day = 13 
  AND rec.userIdentity.invokedBy <> 'dynamodb.application-autoscaling.amazonaws.com'
  AND rec.userIdentity.sessionContext.attributes.mfaAuthenticated IS NOT NULL
LIMIT 25;

## 14. Inline Array Queries - `INNER JOIN`
To query the objects in the nested array we declared it as a table and use an inner join. Note that rows will excluded if the array is empty for that row.

In [ ]:
%%sql
SELECT rec.eventSource
     , log.region 
     , arn.type
     , COUNT(*)
FROM rawlogs.cloudtrail log 
   , log.records rec   -- # Top level inline "Records" array
   , rec.resources arn -- # "Resources" inline table array
WHERE log.log_year = 2019 
  AND log.log_month = 11 
  AND log.log_day = 9
GROUP BY 1,2,3
ORDER BY 1
;

## 15. Inline Array Queries - `OUTER JOIN`
To get all rows, even when the inline array is empty, use a left outer join when declaring the array as a table. If the array is empty that row will not appear.

In [ ]:
%%sql
SELECT rec.eventSource
     , log.region 
     , arn.type
     , COUNT(*)
FROM rawlogs.cloudtrail log 
JOIN log.records rec   ON true     -- # Top level inline "Records" array
LEFT JOIN rec.resources arn ON true -- # "Resources" inline table array
WHERE log.log_year = 2019 
  AND log.log_month = 11 
  AND log.log_day = 9 
GROUP BY 1,2,3
ORDER BY 1
;

## Further Info on AWS CloudTrail Logs
* Redshift Documentation: ["Logging Amazon Redshift API Calls with AWS CloudTrail"](https://docs.aws.amazon.com/redshift/latest/mgmt/db-auditing.html#rs-db-auditing-cloud-trail)
* AWS Documentation: ["Querying AWS CloudTrail Logs"](https://docs.aws.amazon.com/athena/latest/ug/cloudtrail-logs.html)
* AWS Labs (on GitHub): ["AthenaGlueServiceLogs"](https://github.com/awslabs/athena-glue-service-logs)'

